# Deteção de armas e facas

In [ ]:
import os
import numpy as np

from PIL import Image
from keras.utils import np_utils
from keras.models import Sequential
from keras.preprocessing.image import  img_to_array
from sklearn.model_selection import train_test_split
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D

In [ ]:
# input image dimensions
m, n = 240, 240

In [ ]:
train_path = 'data/train/'
test_path = 'data/test/'

In [ ]:
classes = os.listdir(train_path)[1:]
classes

In [ ]:
x=[]
y=[]
count = 0

for folder in classes:
    imgfiles = os.listdir(f'{train_path}{folder}')

    for image in imgfiles:
        if '.DS_Store' != image:
            im = Image.open(f'{train_path}{folder}/{image}')
            im = im.convert(mode='RGB')
            imrs = im.resize((m,n))
            imrs = img_to_array(imrs) / 255
            imrs = imrs.transpose(2, 0, 1)
            imrs = imrs.reshape(3, m, n)
            x.append(imrs)
            y.append(count)

    count += 1

In [ ]:
x=np.array(x)
y=np.array(y)

In [ ]:
batch_size = 32
nb_classes = len(classes)
nb_epoch = 60
nb_filters = 128
nb_pool = 2
nb_conv = 3

In [ ]:
x_train, x_test, y_train, y_test= train_test_split(x,y,test_size=0.2,random_state=4)

In [ ]:
uniques, id_train=np.unique(y_train,return_inverse=True)
Y_train=np_utils.to_categorical(id_train,nb_classes)

In [ ]:
uniques, id_test=np.unique(y_test,return_inverse=True)
Y_test=np_utils.to_categorical(id_test,nb_classes)

In [ ]:
model= Sequential()
model.add(Convolution2D(nb_filters,nb_conv,nb_conv,border_mode='same',input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Convolution2D(int(nb_filters/2),nb_conv,nb_conv,border_mode='same'))
model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(nb_pool,nb_pool)))
model.add(Dropout(0.2))
model.add(Convolution2D(int(nb_filters/4),nb_conv,nb_conv,border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(int(nb_filters/8),nb_conv,nb_conv,border_mode='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(nb_pool,nb_pool)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])

In [ ]:
history = model.fit(
    x_train,
    Y_train,
    batch_size=batch_size,
    nb_epoch=nb_epoch,
    verbose=1,
    validation_data=(x_test, Y_test)
)

In [ ]:
model.save('model/model_latest.h5', overwrite=True)

In [ ]:
image_test = os.listdir(test_path)[0]
im = Image.open(test_path + image_test)
imrs = im.resize((m, n))
imrs = img_to_array(imrs) / 255
imrs = imrs.transpose(2, 0, 1)
imrs = imrs.reshape(3, m, n)

In [ ]:
x = []
x.append(imrs)
x = np.array(x)
predictions = model.predict(x)
predictions